## <center>MPIIO - Distributed File Systems</center>
### <center> Linh B. Ngo </center>
### <center> CPSC 3620 </center>

In [1]:
import ipyparallel
c=ipyparallel.Client(profile="mpicluster")
print(c.ids)

[0, 1, 2, 3]


In [10]:
%%px
from mpi4py import MPI
import numpy as np
    
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
amode = MPI.MODE_WRONLY|MPI.MODE_CREATE
comm = MPI.COMM_WORLD
fh = MPI.File.Open(comm, "./datafile.contig", amode)
   
buffer = np.empty(10, dtype=np.int)
buffer[:] = rank
print(buffer)

offset = comm.Get_rank()*buffer.nbytes
fh.Write_at_all(offset, buffer)  
fh.Close()

if (rank == 0):
    print (np.fromfile("./datafile.contig", dtype="int"))

[stdout:0] [3 3 3 3 3 3 3 3 3 3]
[stdout:1] [1 1 1 1 1 1 1 1 1 1]
[stdout:2] 
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3]
[stdout:3] [2 2 2 2 2 2 2 2 2 2]


In [13]:
%%px
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

amode = MPI.MODE_WRONLY|MPI.MODE_CREATE
fh = MPI.File.Open(comm, "./datafile.noncontig", amode)

item_count = 10

buffer = np.empty(item_count, dtype='i')
buffer[:] = rank
print (buffer)
filetype = MPI.INT.Create_vector(item_count, 1, size)
filetype.Commit()

displacement = MPI.INT.Get_size()*rank
fh.Set_view(displacement, filetype=filetype)

fh.Write_all(buffer)
filetype.Free()
fh.Close()

if (rank == 0):
    print (np.fromfile("./datafile.noncontig", dtype="i"))

[stdout:0] [3 3 3 3 3 3 3 3 3 3]
[stdout:1] [1 1 1 1 1 1 1 1 1 1]
[stdout:2] 
[0 0 0 0 0 0 0 0 0 0]
[0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0
 1 2 3 3 0 0 0 0 0 1 2 3 0 0 0 0 0 1 2 3 0 0 0 0 0 1 2 3 0 0 0 0 0 1 2 3 0
 0 0 0]
[stdout:3] [2 2 2 2 2 2 2 2 2 2]
